# Project 1 Michał Lewandowski

### Imports

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm, lognorm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.express as px
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, root_mean_squared_error, silhouette_score, silhouette_samples, calinski_harabasz_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import statsmodels.api as sm

### Helper functions

In [ ]:
def print_short_classification_report(y_true, y_pred):
    print('accuracy', accuracy_score(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print('confusion matrix')
    print(confusion_matrix(y_true, y_pred))

In [ ]:
def show_feature_importance(feature_importances, column_labels, std=None):
    # model has to be trained first
    plt.barh(column_labels, feature_importances, xerr=std)
    plt.xlabel("Feature Importance")
    plt.show()

In [ ]:
def plot_correlation_matrix(df, width=15, height=12):
    correlation_matrix = df.corr()
    
    plt.figure(figsize=(width, height))
    sns.heatmap(correlation_matrix, annot=True, cmap='vlag', fmt=".2f", linewidths=0.5)
    
    plt.title("Correlation Matrix")
    plt.show()

In [ ]:
def evaluate_KMeans(data, i):
    km = KMeans(n_clusters=i,random_state=0,n_init="auto").fit(data)
    print(f"{i} clusters silhouette: {silhouette_score(data, km.labels_)}, CH score: {calinski_harabasz_score(data, km.labels_)}")

In [ ]:
def evaluate_GMM(data, i, covariance_type='full'):
    gmm = GaussianMixture(n_components=i,random_state=0,n_init=5, covariance_type=covariance_type).fit(data)
    labels = gmm.predict(data)
    print(f"{i} clusters silhouette: {silhouette_score(data, labels)}, CH score: {calinski_harabasz_score(data, labels)}")

In [ ]:
def silhouette_plots_GMM(X, min_cl, max_cl, figsize_x=15, figsize_y=6):
    GAP_BETWEEN_SILHOUETTES = 10
    for n_clusters in range(min_cl, max_cl + 1):
        fig, ax1 = plt.subplots(1, 1)
        fig.set_size_inches(figsize_x, figsize_y)

        ax1.set_xlim([-0.1, 1])
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * GAP_BETWEEN_SILHOUETTES])

        gmm = GaussianMixture(n_components=n_clusters,random_state=0,n_init=5, covariance_type='spherical')
        cluster_labels = gmm.fit_predict(X)

        silhouette_avg = silhouette_score(X, cluster_labels)
        print(
            "For n_clusters =",
            n_clusters,
            "The average silhouette_score is :",
            silhouette_avg,
        )

        sample_silhouette_values = silhouette_samples(X, cluster_labels)

        y_lower = GAP_BETWEEN_SILHOUETTES
        for i in range(n_clusters):
            i_th_scores = sample_silhouette_values[cluster_labels == i]
            i_th_scores.sort()

            i_th_size = i_th_scores.shape[0]
            y_upper = y_lower + i_th_size
            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(
                np.arange(y_lower, y_upper),
                0, 
                i_th_scores,
                facecolor=color,
                edgecolor=color,
                alpha=0.7
            )
            ax1.text(-0.05, y_lower + 0.5 * i_th_size, str(i))
            y_lower = y_upper + GAP_BETWEEN_SILHOUETTES

        ax1.axvline(silhouette_avg, color='red', linestyle='--')

        ax1.set_yticks([])
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

## Task 1. Data description

### Loading data

In [ ]:
raw_data = pd.read_csv("earnings.csv", sep=";")
data = raw_data.drop(columns="id")
print("Total number of observations:", len(raw_data))
print("Total number of columns:", len(raw_data.columns))
print("Total number of features:", len(data.columns))
raw_data.head()

### How many quantitative and how many qualitative variables do we have?

In [ ]:
for id, col in enumerate(raw_data.columns):
    print(col, ':', pd.unique(raw_data.iloc[:, id]))

| Quantitative      | Qualitative |
| ----------------- | ----------- |
| base              | sector      |
| bonus             | section_07  |
| overtime_pay      | sex         |
| other             | education   |
| age               | contract    |
| duration_total    |             |
| duration_entity   |             |
| duration_nominal  |             |
| duration_overtime |             |

There is 9 quantitative and 5 qualitative variables. \
Dataset contains 11000 records.

In [ ]:
qualitative = ['sector', 'section_07', 'sex', 'education', 'contract']
quantitative = data.columns.drop(qualitative).astype(str).tolist()

### Are there any missing data?

In [ ]:
print(raw_data.isnull().sum())

There is no missing data

### Provide and describe appropriate frequency tables or descriptive statistics for the variables.

#### Quantitative data

In [ ]:
data[quantitative].describe()

#### Qualitative data

In [ ]:
map = {}
map['sector'] = {1: 'public', 2: 'private'}
map['section_07'] = {1: 'Public Administration and Defence; Compulsory Social Security', 2: 'Education', 3: 'Human Health and Social Work Activities'}
map['sex'] = {1: 'man', 2: 'woman'}
map['education'] = {1: 'doctorate', 2: 'higher', 3: 'post-secondary', 4: 'secondary', 5: 'basic vocational', 6: 'middle school and below'}
map['contract'] = {1: 'for an indefinite period', 2: 'for a definite period'}

In [ ]:
for col in qualitative:
    tmp = data[col]
    res = tmp.map(map[col])
    print(res.value_counts())
    print()

### Present and discuss (where appropriate) variables’ distributions, e.g. compare them with the normal, or other distribution.

In [ ]:
n_cols = len(quantitative)
fig, axes = plt.subplots(n_rows := n_cols, 1, figsize=(7, 0.7 * n_rows), sharex=False)
if n_cols == 1:
    axes = [axes]
for ax, col in zip(axes, quantitative):
    ax.plot(data[col], [0] * len(data[col]), '|', markersize=30, color='black')
    ax.set_yticks([])
    ax.set_xlim(min(data[col]), max(data[col]))
    ax.set_title(col, fontsize=8, pad=2)
    ax.tick_params(axis='x', labelsize=7)
    ax.grid(True, axis='x', linestyle='--', alpha=0.4)
plt.subplots_adjust(hspace=1.2)
plt.show()

#### Comparison of quantitative variables distributions to normal distribution.

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12,10))
axes = axes.flatten()
plt.tight_layout()

for i, col in enumerate(data[quantitative]):
    ax = axes[i]
    data[col].hist(ax=ax, bins=30, density=True, edgecolor='black')
    mu, sigma = np.mean(data[col]), np.std(data[col])
    x = np.linspace(min(data[col]), max(data[col]), 1000)
    y = norm.pdf(x, mu, sigma)
    ax.plot(x, y, 'r-')
    ax.set_title(col)

>As we can see, *base* fits the normal distribution quite well. As well *age* and *duration_total* somehow can be approximated by the gaussian but this remains a disputed issue. It may seem that *bonus*, *overtime_pay* and *other* are also well described. However we can't determine this due to low resolution of these plots. So let's check more thoroughly.

In [ ]:
def cut_data(data, col, min, max):
    return data[(data[col] > min) & (data[col] <= max)][col]

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,5))
axes = axes.flatten()
plt.tight_layout()

for i, col in enumerate(data[['bonus', 'overtime_pay', 'other']]):
    ax = axes[i]
    part = cut_data(data, col, 0, 22000)
    part.hist(ax=ax, bins=25, density=True, edgecolor='black')
    mu, sigma = np.mean(data[col]), np.std(data[col])
    x = np.linspace(min(part), max(part), 1000)
    y = norm.pdf(x, mu, sigma)
    ax.plot(x, y, 'r-')
    ax.set_title(col)

>After zooming in *bonus* and *overtime_pay* don't fit normal distribution. However *other* appears to be fitting it slightly, but we must take into account that we discarded values that equaled to $0$. Otherwise there would be no match between this variable and gaussian.

>Let's check lognormal distribution.

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12,10))
axes = axes.flatten()
plt.tight_layout()

for i, col in enumerate(data[quantitative]):
    ax = axes[i]
    data[col].hist(ax=ax, bins=30, density=True, edgecolor='black')
    log_data = np.log(data[col][data[col] > 0])  # Ensure values > 0 for log
    mu, sigma = np.mean(log_data), np.std(log_data)
    x = np.linspace(min(data[col]), max(data[col]), 1000)
    y = lognorm.pdf(x, s=sigma, scale=np.exp(mu))
    ax.plot(x, y, 'r-', label='Lognormal fit')
    ax.set_title(col)
    ax.legend()

>Only *other* follows logistic distribution.

>Let's check what is going on with qualitative variables

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12,8))
axes = axes.flatten()

for i, col in enumerate(data[qualitative]):
    ax = axes[i]
    data[col].value_counts(sort=False).plot(kind='bar', ax=ax)
    ax.set_title(col)

>Fitting distribution would make sense when it comes to *education* because the order is defined. However, it doesn't follow any distribution that I am aware of.

## Task. 2 Clustering

>Scaling the data because algorithms like KMeans use distance between points. Since each variable is on its own scale model would be more sensitive to variable with highest variance.

>We have to do OneHotEncoding on unordered features because otherwise it would be misleading for the model.

In [ ]:
ordered = quantitative + ['education']
unordered = list(set(data.columns) - set(ordered))

In [ ]:
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.set_output(transform='pandas')
features_scaled = scaler.fit_transform(data[ordered])

encoder = OneHotEncoder(drop='first', sparse_output=False)
encoder.set_output(transform='pandas')
one_hot_features = encoder.fit_transform(data[unordered])

data_scaled = pd.concat([features_scaled, one_hot_features], axis=1)  
data_scaled.head()

Let's try different approaches.

>Clustering metrics are not only sensitive to dataset but also to selected features. This applies directly to our dataset:

>Clustering whole dataset:

In [ ]:
for i in range(2, 10):
    evaluate_KMeans(data_scaled, i)

>Clustering two arbitrary chosen features:

In [ ]:
for i in range(2, 10):
    evaluate_KMeans(data_scaled[['duration_total', 'sex_2']], i)

>There’s little value in clustering just to improve evaluation scores.

>We can try dimensionality reduction and examine results visually.

In [ ]:
pca = PCA(n_components=3)
pca.set_output(transform='pandas')
reduced = pca.fit_transform(data_scaled)
reduced.head()

In [ ]:
fig = px.scatter_3d(reduced, x='pca0', y='pca1', z='pca2')
fig.update_traces(marker=dict(size=1))
fig.show()

>It is hard to determine whether there are clusters. I can see something (plate and very closely a corner shape) but it can be only seen from a certain angle and is not clear.

> Let's try clustering it by KMeans.

In [ ]:
reduced_clustered = reduced.copy()
reduced_clustered['cluster'] = KMeans(n_clusters=2).fit_predict(reduced)
fig = px.scatter_3d(reduced_clustered, x='pca0', y='pca1', z='pca2', color='cluster')
fig.update_traces(marker=dict(size=1))
fig.show()

>Unfortunately KMeans didn't pick up those clusters that i have seen😥. Maybe it was an illusion.

> Let's try GaussianMixture model.

In [ ]:
reduced_clustered = reduced.copy()
reduced_clustered['cluster'] = GaussianMixture(n_components=2, n_init=10).fit_predict(reduced)
fig = px.scatter_3d(reduced_clustered, x='pca0', y='pca1', z='pca2', color='cluster')
fig.update_traces(marker=dict(size=1))
fig.show()

>Maybe I wasn't wrong. Gaussian model almost identified what I had in mind.

>But still they are very unclear. We can't be fully sure. We should plot all possible pairwise plots to get better understanding of the data.

In [ ]:
sns.pairplot(data_scaled)
plt.show()

>There are no visible clusters. \
>We will train multiple models and pick one that performs the best, and then analyze it further.

In [ ]:
for i in range(2, 10):
    evaluate_GMM(data_scaled, i, 'full')

In [ ]:
for i in range(2, 10):
    evaluate_GMM(data_scaled, i, 'tied')

In [ ]:
for i in range(2, 10):
    evaluate_GMM(data_scaled, i, 'diag')

In [ ]:
for i in range(2, 10):
    evaluate_GMM(data_scaled, i, 'spherical')

>Even though KMeans perfomed better than Gaussian Mixture model. I will stick to GMM just because it detected my illusive vision. \
>I will choose covariance_type='spherical' becaused it has the best scores across other GMMs.

In [ ]:
silhouette_plots_GMM(data_scaled, 2, 9)

> Highest silhouette_score was for 3 clusters. We should check how would this model cluster the data.

In [ ]:
reduced_clustered = reduced.copy()
reduced_clustered['cluster'] = GaussianMixture(n_components=3, n_init=10, covariance_type='spherical').fit_predict(data_scaled)
fig = px.scatter_3d(reduced_clustered, x='pca0', y='pca1', z='pca2', color='cluster')
fig.update_traces(marker=dict(size=1))
fig.show()

In [ ]:
data_scaled_clustered = data_scaled.copy()
data_scaled_clustered['cluster'] = GaussianMixture(n_components=3, n_init=10, covariance_type='spherical').fit_predict(data_scaled)
sns.pairplot(data_scaled_clustered, hue='cluster')
plt.show()

>Therefore, I’d say that our data is not naturally clusterable—or if it is, the structure is difficult to observe and these techniques don’t yield any meaningful insights. However, it made me realize the importance of always asking: what is the goal of clustering?

>Purely out of curiosity, let's take a look at how my elusive clustering appears on a feature pair plot.

In [ ]:
data_scaled_clustered = data_scaled.copy()
data_scaled_clustered['cluster'] = GaussianMixture(n_components=2, n_init=10).fit_predict(reduced)
sns.pairplot(data_scaled_clustered, hue='cluster')
plt.show()

## Task 3. Classification

#### Convert data
1 indicates higher education (<=2) \
0 otherwise

In [ ]:
X = data_scaled.drop(columns='education')
y = (data['education'] <= 2).astype(int)

>Try different models

### Logistic regression

In [ ]:
model = LogisticRegression()
y_pred = cross_val_predict(model, X, y, cv=5)

print_short_classification_report(y, y_pred)

model.fit(X, y)

feature_importances = model.coef_[0]
show_feature_importance(feature_importances, X.columns)

### Bagging classifier

In [ ]:
model = BaggingClassifier(DecisionTreeClassifier())
y_pred = cross_val_predict(model, X, y, cv=5)

print_short_classification_report(y, y_pred)
model.fit(X, y)
feature_importances = np.mean([tree.feature_importances_ for tree in model.estimators_], axis=0)
show_feature_importance(feature_importances, X.columns)

### Random forest classifier

In [ ]:
model = RandomForestClassifier()
y_pred = cross_val_predict(model, X, y, cv=5)

print_short_classification_report(y, y_pred)

model.fit(X, y)
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
show_feature_importance(model.feature_importances_, X.columns, std)

In [ ]:
RFC_accuracies = cross_val_score(model, X, y, cv=5)

### AdaBoost classifier

In [ ]:
model = AdaBoostClassifier()
y_pred = cross_val_predict(model, X, y, cv=5)

print_short_classification_report(y, y_pred)

model.fit(X, y)
show_feature_importance(model.feature_importances_, X.columns)

>Thanks to `cross_val_predict` we were able to do cross validation and predict label for each data point as if that data point had been in the test set of a cross-validation fold.

>As we saw before the model that performed the best was RandomForestClassifier.

In [ ]:
plt.bar(range(1, len(RFC_accuracies) + 1), RFC_accuracies)
plt.title('Comparison of RandomForestClassifier accuracies across folds')
plt.show()

>Based on accuracies across folds we may predict that model is stable and has not been overfitted, and will perform quite well on unseen data.

>At the feature importance plot of RandomForestClassifier we can see that *base* and *duration_nominal* are the most significant variables. However when we look at feature importance plot of LogisticRegression we may see that some featues have negative importance, which indicates that they have negative influence on the final prediction. In this case, the higher *duration_nominal* the more likely it is that predicted education degree will be worse(>2). Absolute values of feature importances of both models look somewhat similar so we may suspect that RandomForestClassifier has the same dependency.

## Task 4. Regression

Based on the task description, we can assume that the model should be highly interpretable, which strongly suggests using linear regression.

#### Data preparation

>We use scaled data in order to preserve numerical stability.

In [ ]:
X = data_scaled.drop(columns='base')
y = data['base']
X_train_1, X_test_1, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train_1 = sm.tools.tools.add_constant(X_train_1)
X_test_1 = sm.tools.tools.add_constant(X_test_1)

>Train first model on all features.

In [ ]:
model_1 = sm.regression.linear_model.OLS(y_train, X_train_1)
res_1 = model_1.fit()
res_1.summary()

In [ ]:
pred = res_1.predict(X_test_1)
print('rmse:', root_mean_squared_error(pred, y_test))

>Only *sector* variable has t-value lower than 2 which indicates its insignificance. Based on table above we should discard only this feature. However it is worth looking into correlation matrix.

In [ ]:
plot_correlation_matrix(X_train_1)

>We can see that *duration_total* and *age* are highly correlated as well as *duration_overtime* and *overtime_pay*. We expect that after removing one variable from each pair, our model's performance won't deteriorate significantly. Let's remove *duration_overtime* and *age*.

In [ ]:
X_train_2 = X_train_1.drop(columns=['duration_overtime', 'age', 'sector_2'])
X_test_2 = X_test_1.drop(columns=['duration_overtime', 'age', 'sector_2'])

In [ ]:
model_2 = sm.regression.linear_model.OLS(y_train, X_train_2)
res_2 = model_2.fit()
res_2.summary()

In [ ]:
pred = res_2.predict(X_test_2)
print('rmse:', root_mean_squared_error(pred, y_test))

>After deletion of three variables out model perform slightly worse than before. We can try deleting 2 more variables.

In [ ]:
X_train_3 = X_train_2.drop(columns=['section_07_3', 'sex_2'])
X_test_3 = X_test_2.drop(columns=['section_07_3', 'sex_2'])

In [ ]:
model_3 = sm.regression.linear_model.OLS(y_train, X_train_3)
res_3 = model_3.fit()
res_3.summary()

In [ ]:
pred = res_3.predict(X_test_3)
print('rmse:', root_mean_squared_error(pred, y_test))

>There was a slight decline in prediction accuracy. It was so small that it can be omittable. Let's go even futher and delete 2 more variables.

In [ ]:
X_train_4 = X_train_3.drop(columns=['duration_entity', 'contract_2'])
X_test_4 = X_test_3.drop(columns=['duration_entity', 'contract_2'])

In [ ]:
model_4 = sm.regression.linear_model.OLS(y_train, X_train_4)
res_4 = model_4.fit()
res_4.summary()

In [ ]:
pred = res_4.predict(X_test_4)
print('rmse:', root_mean_squared_error(pred, y_test))

>Both RMSE and R squared metric decreased notably. So we will stick to the previous model no. 3.

In [ ]:
res_3.summary()

>Features used in model_3: \
bonus - moderate positive influence - increase in bonus increases salary \
overtime_pay - more significant then bonus in the same way \
other - highest positive influence \
duration_total - very high positive influence \
duration_entity - lowest positive influence across all variables, barerly noticable \
duration_nominal - slightly lower positive influence than other variable \
education - strongest negative impact. But since education levels were inverted the lower the education the variable the higher the total income. It is the most significant variable in our model. \
section_07_2 - negative impact, but still quite noticable
contract_2	- negative coefficient, doesn't affect model that much